In [4]:
import pandas as pd
import numpy as np
import re
import ast
import os
import warnings
warnings.filterwarnings('ignore')

# Create output directory if it doesn't exist
output_dir = 'csv_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Loading data...")
# Read the data
df = pd.read_csv('raw_anime.csv')

# -----------------
# Data Cleaning
# -----------------
print("Cleaning data...")

# Handle missing values
df['score'] = pd.to_numeric(df['score'], errors='coerce')
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

# Extract year from aired field
def extract_year(aired_str):
    if pd.isna(aired_str):
        return np.nan
    try:
        match = re.search(r"'year': (\d{4})", str(aired_str))
        if match:
            return int(match.group(1))
        return np.nan
    except:
        return np.nan

df['start_year'] = df['aired'].apply(extract_year)

# Process genres - convert string representation of list to actual list
def parse_genres(genre_str):
    if pd.isna(genre_str) or genre_str == '[]':
        return []
    try:
        genres = ast.literal_eval(genre_str)
        return genres
    except:
        clean_str = genre_str.replace('[', '').replace(']', '').replace("'", "")
        return [g.strip() for g in clean_str.split(',') if g.strip()]

df['genres'] = df['genre'].apply(parse_genres)

# Process duration - extract minutes
def extract_minutes(duration_str):
    if pd.isna(duration_str):
        return np.nan
    try:
        if 'hr' in duration_str:
            hr_match = re.search(r'(\d+)\s*hr', duration_str)
            min_match = re.search(r'(\d+)\s*min', duration_str)
            hours = int(hr_match.group(1)) if hr_match else 0
            minutes = int(min_match.group(1)) if min_match else 0
            return hours * 60 + minutes
        elif 'per ep' in duration_str:
            match = re.search(r'(\d+)\s*min', duration_str)
            if match:
                return int(match.group(1))
        else:
            match = re.search(r'(\d+)\s*min', duration_str)
            if match:
                return int(match.group(1))
        return np.nan
    except:
        return np.nan

df['duration_minutes'] = df['duration'].apply(extract_minutes)

# Calculate total runtime (episodes * duration)
df['total_runtime'] = df.apply(
    lambda row: row['duration_minutes'] * row['episodes'] if not pd.isna(row['episodes']) and row['episodes'] > 0 
    else row['duration_minutes'], 
    axis=1
)

# Filter out very long anime or ones with missing data
df_filtered = df[
    (df['score'].notna()) & 
    (df['start_year'].notna()) &
    (df['start_year'] >= 1970) &
    (df['start_year'] <= 2020)
]

# Create a version with runtime filter applied
df_runtime = df_filtered[
    (df['total_runtime'].notna()) &
    (df['total_runtime'] <= 5000)
]

print(f"Total anime records: {len(df)}")
print(f"Records after filtering: {len(df_filtered)}")
print(f"Records with valid runtime data: {len(df_runtime)}")

# -----------------
# Generate CSV Files for Visualizations
# -----------------
print("\nGenerating filtered CSV files...")

# 1. Genre Trends Over Time
df_genres = df_filtered.explode('genres')
df_genres = df_genres[df_genres['genres'].notna() & (df_genres['genres'] != '')]
top_genres = df_genres['genres'].value_counts().head(10).index.tolist()

genre_trend_data = []
for year in sorted(df_genres['start_year'].unique()):
    year_total = df_genres[df_genres['start_year'] == year].shape[0]
    if year_total >= 10:
        for genre in top_genres:
            genre_count = df_genres[(df_genres['start_year'] == year) & 
                                   (df_genres['genres'] == genre)].shape[0]
            percentage = (genre_count / year_total) * 100
            genre_trend_data.append({
                'Year': year,
                'Genre': genre,
                'Percentage': percentage,
                'Count': genre_count
            })

genre_trends_df = pd.DataFrame(genre_trend_data)
genre_trends_df.to_csv(os.path.join(output_dir, 'genre_trends_over_time.csv'), index=False)

# 2. Average Ratings by Genre Over Time
genre_score_data = []
for year in sorted(df_genres['start_year'].unique()):
    year_data = df_genres[df_genres['start_year'] == year]
    if len(year_data) >= 10:
        for genre in top_genres:
            genre_year_data = year_data[year_data['genres'] == genre]
            if len(genre_year_data) >= 5:
                avg_score = genre_year_data['score'].mean()
                genre_score_data.append({
                    'Year': year,
                    'Genre': genre,
                    'Average Score': avg_score,
                    'Count': len(genre_year_data)
                })

genre_scores_df = pd.DataFrame(genre_score_data)
genre_scores_df.to_csv(os.path.join(output_dir, 'average_genre_ratings_over_time.csv'), index=False)

# 3. Genre Ratings by Decade (Heatmap Data)
df_genres['decade'] = (df_genres['start_year'] // 10) * 10
genre_decade_scores = df_genres.groupby(['decade', 'genres'])['score'].agg(['mean', 'count']).reset_index()
genre_decade_scores = genre_decade_scores[
    (genre_decade_scores['count'] >= 10) &
    (genre_decade_scores['genres'].isin(top_genres))
]
genre_decade_scores.rename(columns={'mean': 'average_score'}, inplace=True)
genre_decade_scores.to_csv(os.path.join(output_dir, 'average_genre_ratings_by_decade.csv'), index=False)

# 4. Score by Episode Duration
duration_bins = [0, 15, 25, 35, 60, float('inf')]
duration_labels = ['< 15 min', '15-24 min', '25-34 min', '35-60 min', '> 60 min']
df_duration = df_filtered[df_filtered['duration_minutes'].notna()].copy()
df_duration['duration_category'] = pd.cut(df_duration['duration_minutes'], bins=duration_bins, labels=duration_labels, right=False)
duration_scores = df_duration.groupby('duration_category')['score'].agg(['mean', 'count', 'std']).reset_index()
duration_scores = duration_scores[duration_scores['count'] > 20]
duration_scores.rename(columns={'mean': 'average_score'}, inplace=True)
duration_scores['error'] = 1.96 * duration_scores['std'] / np.sqrt(duration_scores['count'])
duration_scores.to_csv(os.path.join(output_dir, 'score_by_episode_duration.csv'), index=False)

# 5. Score by Anime Type
type_scores = df_filtered.groupby('type')['score'].agg(['mean', 'count', 'std']).reset_index()
type_scores = type_scores[type_scores['count'] > 20]
type_scores.rename(columns={'mean': 'average_score'}, inplace=True)
type_scores['error'] = 1.96 * type_scores['std'] / np.sqrt(type_scores['count'])
type_scores.to_csv(os.path.join(output_dir, 'score_by_anime_type.csv'), index=False)

# 6. Score Distribution by Episode Count
df_ep_box = df_filtered[df_filtered['episodes'] <= 100].copy()
df_ep_box['episode_group'] = pd.cut(df_ep_box['episodes'], 
                                   bins=[0, 1, 6, 13, 26, 52, 100],
                                   labels=['1', '2-6', '7-13', '14-26', '27-52', '53-100'])
df_ep_box[['name', 'score', 'episodes', 'episode_group', 'type']].to_csv(
    os.path.join(output_dir, 'score_distribution_by_episode_count.csv'), index=False)

# 7. Runtime vs Score by Type
df_runtime[['name', 'score', 'total_runtime', 'type', 'episodes', 'duration_minutes']].to_csv(
    os.path.join(output_dir, 'runtime_vs_score_by_type.csv'), index=False)

print(f"All relevant CSVs saved to the '{output_dir}/' directory.")

Loading data...
Cleaning data...
Total anime records: 15278
Records after filtering: 7125
Records with valid runtime data: 6878

Generating filtered CSV files...
All relevant CSVs saved to the 'csv_outputs/' directory.
